Ontology Design (Concepts + Relationships)
Concepts (Classes)

Student

Course

Relationships (Properties)

requires(Course → Course): defines prerequisite courses

completed(Student → Course): records courses completed by a student

eligibleFor(Student → Course): inferred relationship

Explanation:
The ontology structures academic advising knowledge by defining what entities exist and how they are connected.
Inference rules operate on these relationships to determine eligibility and recommendations.

Implementation (Class + Methods)

In [50]:
#kbs.py class

class UniversityOntologyKBS:
    def __init__(self):
        self.students = set()
        self.courses = set()
        self.prerequisites = {}     # course -> set(prereq courses)
        self.completed = {}         # student -> set(completed courses)

    #Add entities
    def add_student(self, student: str):    #add_student(student: str)
        self.students.add(student)
        self.completed.setdefault(student, set())

    def add_course(self, course: str): #add_course(course: str)
        self.courses.add(course)
        self.prerequisites.setdefault(course, set())

    def add_prerequisite(self, course: str, prereq: str): #add_prerequisite(course: str, prereq: str)
        if course not in self.courses:
            raise ValueError(f"Unknown course: {course}")
        if prereq not in self.courses:
            raise ValueError(f"Unknown course: {prereq}")
        self.prerequisites[course].add(prereq)

    def complete_course(self, student: str, course: str): #complete_course(student: str, course: str)
        if student not in self.students:
            raise ValueError(f"Unknown student: {student}")
        if course not in self.courses:
            raise ValueError(f"Unknown course: {course}")
        self.completed[student].add(course)

    #Inference 
    def can_take(self, student: str, course: str): #can_take(student: str, course: str) -> (bool, set)
        if student not in self.students:
            raise ValueError(f"Unknown student: {student}")
        if course not in self.courses:
            raise ValueError(f"Unknown course: {course}")

        required = self.prerequisites.get(course, set())
        done = self.completed.get(student, set())

        missing = required - done

        if len(missing) == 0:
            return True, set()
        else:
            return False, missing

    def recommend_courses(self, student: str): #recommend_courses(student: str) -> list
        if student not in self.students:
            raise ValueError(f"Unknown student: {student}")

        recommendations = []

        for course in self.courses:
            # Exclude completed courses
            if course in self.completed[student]:
                continue

            eligible, _ = self.can_take(student, course)
            if eligible:
                recommendations.append(course)

        return sorted(recommendations)


Dataset (Courses, Students, Prerequisites)

In [51]:


def main():
    kb = UniversityOntologyKBS()

    # Add Courses
    courses = [
        "Intro Math",
        "Statistics",
        "Python",
        "Machine Learning",
        "AI",
        "Databases"
    ]

    for c in courses:
        kb.add_course(c)

    # Add Students 3 students
    students = ["Alice", "Bob", "Carol"]
    for s in students:
        kb.add_student(s)

    # Add Prerequisites
    kb.add_prerequisite("Statistics", "Intro Math")
    kb.add_prerequisite("Machine Learning", "Statistics")
    kb.add_prerequisite("Machine Learning", "Python")
    kb.add_prerequisite("AI", "Machine Learning")
    kb.add_prerequisite("AI", "Statistics")
    kb.add_prerequisite("Databases", "Python")
    kb.add_prerequisite("Databases", "Statistics")

    # Completed Courses
    # Alice: fully prepared for AI
    kb.complete_course("Alice", "Intro Math")
    kb.complete_course("Alice", "Statistics")
    kb.complete_course("Alice", "Python")
    kb.complete_course("Alice", "Machine Learning")

    # Bob: partially prepared
    kb.complete_course("Bob", "Intro Math")
    kb.complete_course("Bob", "Python")

    # Carol: barely prepared
    kb.complete_course("Carol", "Intro Math")

    
   


Inference Demo (eligibility checks + explanations)

In [52]:
 # Eligibility Checks
if 'kb' not in globals():
    kb = UniversityOntologyKBS()
    # Add Courses
    courses = [
        "Intro Math",
        "Statistics",
        "Python",
        "Machine Learning",
        "AI",
        "Databases"
    ]
    for c in courses:
        kb.add_course(c)
    students = ["Alice", "Bob", "Carol"]
    for s in students:
        kb.add_student(s)
    kb.add_prerequisite("Statistics", "Intro Math")
    kb.add_prerequisite("Machine Learning", "Statistics")
    kb.add_prerequisite("Machine Learning", "Python")
    kb.add_prerequisite("AI", "Machine Learning")
    kb.add_prerequisite("AI", "Statistics")
    kb.add_prerequisite("Databases", "Python")
    kb.add_prerequisite("Databases", "Statistics")
    kb.complete_course("Alice", "Intro Math")
    kb.complete_course("Alice", "Statistics")
    kb.complete_course("Alice", "Python")
    kb.complete_course("Alice", "Machine Learning")
    kb.complete_course("Bob", "Intro Math")
    kb.complete_course("Bob", "Python")
    kb.complete_course("Carol", "Intro Math")

print("ELIGIBILITY CHECKS\n")
print("Alice -> AI:", kb.can_take("Alice", "AI"))
print("Bob -> AI:", kb.can_take("Bob", "AI"))
print("Carol -> AI:", kb.can_take("Carol", "AI"))


ELIGIBILITY CHECKS

Alice -> AI: (True, set())
Bob -> AI: (False, {'Statistics', 'Machine Learning'})
Carol -> AI: (False, {'Statistics', 'Machine Learning'})


Recommendations

In [53]:
    # Recommendations-
print("\nRECOMMENDATIONS\n")

print("Recommendations for Alice:", kb.recommend_courses("Alice"))
print("Recommendations for Bob:", kb.recommend_courses("Bob"))
print("Recommendations for Carol:", kb.recommend_courses("Carol"))

if __name__ == "__main__":
    main()


RECOMMENDATIONS

Recommendations for Alice: ['AI', 'Databases']
Recommendations for Bob: ['Statistics']
Recommendations for Carol: ['Python', 'Statistics']


Unit Tests (unittest)

In [54]:
import unittest

class TestUniversityOntologyKBS(unittest.TestCase):

    def setUp(self):
        self.kb = UniversityOntologyKBS()
        self.kb.add_course("Math")
        self.kb.add_course("Stats")
        self.kb.add_course("ML")
        self.kb.add_student("Ann")
        self.kb.add_prerequisite("Stats", "Math")
        self.kb.add_prerequisite("ML", "Stats")

    def test_add_student(self):
        self.assertIn("Ann", self.kb.students)

    def test_add_course(self):
        self.assertIn("Math", self.kb.courses)

    def test_complete_course(self):
        self.kb.complete_course("Ann", "Math")
        self.assertIn("Math", self.kb.completed["Ann"])

    def test_can_take_false(self):
        eligible, missing = self.kb.can_take("Ann", "Stats")
        self.assertFalse(eligible)
        self.assertEqual(missing, {"Math"})

    def test_can_take_true(self):
        self.kb.complete_course("Ann", "Math")
        eligible, _ = self.kb.can_take("Ann", "Stats")
        self.assertTrue(eligible)

    def test_recommendations(self):
        self.kb.complete_course("Ann", "Math")
        recs = self.kb.recommend_courses("Ann")
        self.assertIn("Stats", recs)

unittest.main(argv=[''], exit=False)


......
----------------------------------------------------------------------
Ran 6 tests in 0.008s

OK
